In [1]:
import pandas as pd

# Trata os dados para serem usados na predição de percentual de propostas legislativas em favor de grupos historicamente marginalizados quando a composição da Câmera dos Deputados atingir 30% de mulheres

Notebook com predição: [../prever-percentual-proposicoes-beneficentes-grupos-historicamente-marginalizados.ipynb](../prever-percentual-proposicoes-beneficentes-grupos-historicamente-marginalizados.ipynb)

In [2]:
df_candidaturas_eleitas = pd.read_csv('../dados/candidaturas_eleitas.csv')
df_legislaturas = pd.read_csv('../dados/legislaturas_1934_2023_limpas.csv')

In [3]:
df_legislaturas.tail()

,id,uri,dataInicio,dataFim
16,40,https://dadosabertos.camara.leg.br/api/v2/legi...,1955,1959
17,39,https://dadosabertos.camara.leg.br/api/v2/legi...,1951,1955
18,38,https://dadosabertos.camara.leg.br/api/v2/legi...,1946,1951
19,37,https://dadosabertos.camara.leg.br/api/v2/legi...,1935,1937
20,36,https://dadosabertos.camara.leg.br/api/v2/legi...,1934,1935


In [4]:
df_candidaturas_eleitas.head()

,id,siglaPartido,siglaUf,idLegislatura,sexo
0,215044,PSOL,PA,56,F
1,178862,REPUBLICANOS,BA,56,F
2,204466,PSDB,AL,56,F
3,178901,UNIÃO,MS,56,F
4,204464,PSOL,RJ,56,F


Relaciona ano de inicio e fim de cada legislatura ao dataframe com candidaturas eleitas

In [5]:
def relaciona_id_legislatura_ano_inicio(idLegislatura):
    for id, ano in zip(df_legislaturas['id'], df_legislaturas['dataInicio']):
        if idLegislatura == id:
            return ano

In [6]:
def relaciona_id_legislatura_ano_fim(idLegislatura):
    for id, ano in zip(df_legislaturas['id'], df_legislaturas['dataFim']):
        if idLegislatura == id:
            return ano

In [7]:
df_candidaturas_eleitas['ano_inicio'] = df_candidaturas_eleitas['idLegislatura'].apply(relaciona_id_legislatura_ano_inicio)
df_candidaturas_eleitas['ano_fim'] = df_candidaturas_eleitas['idLegislatura'].apply(relaciona_id_legislatura_ano_fim)

In [8]:
df_candidaturas_eleitas.head()

,id,siglaPartido,siglaUf,idLegislatura,sexo,ano_inicio,ano_fim
0,215044,PSOL,PA,56,F,2019,2023
1,178862,REPUBLICANOS,BA,56,F,2019,2023
2,204466,PSDB,AL,56,F,2019,2023
3,178901,UNIÃO,MS,56,F,2019,2023
4,204464,PSOL,RJ,56,F,2019,2023


Agrupa quantidade de mulheres eleitas em cada legislatura

In [9]:
agrupa_sexo = df_candidaturas_eleitas.groupby(['ano_inicio', 'ano_fim', 'sexo']).size().to_frame('valorAbsoluto')

In [10]:
agrupa_sexo.reset_index(inplace=True)

In [36]:
agrupa_sexo.head()e marca e marca 

,ano_inicio,ano_fim,sexo,valorAbsoluto,porcentagem
0,1934,1935,F,1,0.38
1,1935,1937,F,2,0.70
2,1936,1937,F,2,0.70
3,1951,1955,F,1,0.25
4,1952,1955,F,1,0.25


Insere coluna `porcentagem` de mulheres em cada legislatura

In [12]:
agrupa_sexo['porcentagem'] = round(agrupa_sexo['valorAbsoluto'].div(agrupa_sexo.groupby('ano_inicio')['valorAbsoluto'].transform('sum')).mul(100), 2)

In [13]:
agrupa_sexo = agrupa_sexo[agrupa_sexo['sexo'] == 'F'].copy().reset_index(drop=True)

In [14]:
agrupa_sexo.head()

,ano_inicio,ano_fim,sexo,valorAbsoluto,porcentagem
0,1934,1935,F,1,0.38
1,1935,1937,F,2,0.70
2,1951,1955,F,1,0.25
3,1955,1959,F,2,0.45
4,1959,1963,F,1,0.22


Insere período completo de cada legislatura, levando em conta que a proporcionalidade de gênero só é alterada a cada nova eleição.

In [15]:
for ano_inicial, ano_final in zip(agrupa_sexo['ano_inicio'], agrupa_sexo['ano_fim']):
    valorAbsoluto = agrupa_sexo['valorAbsoluto'][agrupa_sexo['ano_inicio'] == ano_inicial].item()
    porcentagem = agrupa_sexo['porcentagem'][agrupa_sexo['ano_inicio'] == ano_inicial].item()
    periodo_legislativo = ano_final-ano_inicial
    if periodo_legislativo > 1:
        for ano in range(1, periodo_legislativo):
            dados = pd.DataFrame(data={
                'ano_inicio': [ano_inicial+ano],
                'ano_fim': ano_final,
                'sexo': ['F'],
                'valorAbsoluto': valorAbsoluto,
                'porcentagem': porcentagem})
            agrupa_sexo = pd.concat([agrupa_sexo, dados])

In [16]:
agrupa_sexo.sort_values(by=['ano_inicio'], inplace=True)

In [17]:
agrupa_sexo.reset_index(drop=True, inplace=True)

In [18]:
agrupa_sexo.head()

,ano_inicio,ano_fim,sexo,valorAbsoluto,porcentagem
0,1934,1935,F,1,0.38
1,1935,1937,F,2,0.70
2,1936,1937,F,2,0.70
3,1951,1955,F,1,0.25
4,1952,1955,F,1,0.25


Agora, vamos agrupar por ano a quantidade de propostas legislativas referente a proteção de direitos de grupos historicamente marginalizados e posteriormente relacionar com porcentagem de mulheres eleitas em cada legislatura.

In [19]:
proposicoes = pd.read_csv('../dados/proposicoes_legislativas_limpas_vocabulario.csv')

In [20]:
proposicoes.head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,temas
0,168293,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d...",NaN
1,168297,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,NaN,"manutenção,ato,tribunal,contas,união,tcu,recus...",NaN
2,168300,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d...",NaN
3,168303,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,NaN,"manutenção,ato,tribunal,contas,união,tcu,negaç...",NaN
4,168307,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,NaN,"manutenção,ato,tribunal,contas,união,tcu,deneg...",NaN


In [21]:
proposicoes_anuais = proposicoes[['ano', 'temas']].groupby(by=['ano']).agg(['size', 'count']).droplevel(0,axis=1).reset_index(col_level=1)

In [22]:
proposicoes_anuais.rename(columns={'size':'total_proposicoes', 'count':'proposicoes_beneficiam_grupos_historicamente_marginalizados'}, inplace=True)

In [23]:
proposicoes_anuais.head()

,ano,total_proposicoes,proposicoes_beneficiam_grupos_historicamente_marginalizados
0,1934,1,0
1,1945,1,0
2,1946,310,7
3,1947,1294,25
4,1948,1572,36


Insere coluna com porcentagem de propostas legislativas referente a proteção de direitos de grupos historicamente marginalizados.

In [24]:
proposicoes_anuais['proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem'] = round(
    proposicoes_anuais['proposicoes_beneficiam_grupos_historicamente_marginalizados'].div(proposicoes_anuais['total_proposicoes']).mul(100), 2)

In [25]:
proposicoes_anuais.head()

,ano,total_proposicoes,proposicoes_beneficiam_grupos_historicamente_marginalizados,proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem
0,1934,1,0,0.00
1,1945,1,0,0.00
2,1946,310,7,2.26
3,1947,1294,25,1.93
4,1948,1572,36,2.29


Insere porcentagem de mulheres em cada legislatura no dataframe `proposicoes_anuais`

In [26]:
def insere_coluna_porcentagem_mulheres_eleitas(ano):
    for ano_sexo, porcentagem in zip(agrupa_sexo['ano_inicio'], agrupa_sexo['porcentagem']):
        if ano_sexo == ano:
            return porcentagem

In [27]:
proposicoes_anuais['mulheres_eleitas_porcentagem'] = proposicoes_anuais['ano'].apply(insere_coluna_porcentagem_mulheres_eleitas)

In [28]:
proposicoes_anuais = proposicoes_anuais.loc[:, ['ano', 'mulheres_eleitas_porcentagem', 'total_proposicoes',
       'proposicoes_beneficiam_grupos_historicamente_marginalizados',
       'proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem']]

In [29]:
proposicoes_anuais.head()

,ano,mulheres_eleitas_porcentagem,total_proposicoes,proposicoes_beneficiam_grupos_historicamente_marginalizados,proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem
0,1934,0.38,1,0,0.00
1,1945,NaN,1,0,0.00
2,1946,NaN,310,7,2.26
3,1947,NaN,1294,25,1.93
4,1948,NaN,1572,36,2.29


Verificae existencia de valores faltantes e investigar dados das legislaturas

In [30]:
proposicoes_anuais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 5 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   ano                                                                      78 non-null     int64  
 1   mulheres_eleitas_porcentagem                                             72 non-null     float64
 2   total_proposicoes                                                        78 non-null     int64  
 3   proposicoes_beneficiam_grupos_historicamente_marginalizados              78 non-null     int64  
 4   proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem  78 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 3.2 KB


In [31]:
proposicoes_anuais[proposicoes_anuais['mulheres_eleitas_porcentagem'].isna()]

,ano,mulheres_eleitas_porcentagem,total_proposicoes,proposicoes_beneficiam_grupos_historicamente_marginalizados,proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem
1,1945,NaN,1,0,0.00
2,1946,NaN,310,7,2.26
3,1947,NaN,1294,25,1.93
4,1948,NaN,1572,36,2.29
5,1949,NaN,1484,41,2.76
6,1950,NaN,1258,32,2.54


In [32]:
df_candidaturas_eleitas[df_candidaturas_eleitas['ano_inicio']==1946].groupby(by=['sexo']).count()

,id,siglaPartido,siglaUf,idLegislatura,ano_inicio,ano_fim
sexo,,,,,,
M,379,379,379,379,379,379


Percebemos que existiram legislaturas sem mulheres eleitas, por isso os valores faltantes na coluna `mulheres_eleitas_porcentagem` será preenchido por 0 (zero)

In [33]:
proposicoes_anuais['mulheres_eleitas_porcentagem'].fillna(value=0, inplace=True)

In [34]:
proposicoes_anuais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 5 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   ano                                                                      78 non-null     int64  
 1   mulheres_eleitas_porcentagem                                             78 non-null     float64
 2   total_proposicoes                                                        78 non-null     int64  
 3   proposicoes_beneficiam_grupos_historicamente_marginalizados              78 non-null     int64  
 4   proposicoes_beneficiam_grupos_historicamente_marginalizados_porcentagem  78 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 3.2 KB


In [35]:
proposicoes_anuais.to_csv('../dados/proposicoes_anuais_para_predicao.csv', index=False)